In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import bloscpack as bp
import matplotlib.pyplot as plt
%matplotlib inline
import vtk
from vtk.util import numpy_support

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *
from vis3d_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for Local Macbook Pro


Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


In [2]:
stack = 'MD589'

In [3]:
contour_df = DataManager.load_annotation_v4(stack=stack, by_human=True, timestamp='latest', suffix='contours')
contour_df = convert_annotation_v3_original_to_aligned(contour_df, stack=stack)
# contour_df vertices wrt to aligned uncropped lossless image

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/home/yuncong/CSHL_labelings_v3/MD589" --exclude "*" --include "*contours*"
latest timestamp:  10042017100807


In [7]:
name_s = '7N_L'
name_u, side = parse_label(name_s)[:2]

contours_xysec_wrt_wholebrain_rawResol = [[(x, y, cnt['section']) for x, y in cnt['vertices']]
 for cid, cnt in contour_df[(contour_df['name'] == name_u) & (contour_df['side'] == side)].iterrows()]

contours_xyz_wrt_wholebrain_volResol = \
[convert_frame_and_resolution(cnt_xysec,
                              in_wrt='wholebrain', in_resolution='image_image_section',
                              out_wrt='wholebrain', out_resolution='volume',
                             image_resolution='raw', stack=stack,
                             volume_resolution_um=10.)
 for cnt_xysec in contours_xysec_wrt_wholebrain_rawResol]

volume_volResol, origin_wrt_wholebrain_volResol = \
interpolate_contours_to_volume(contours_xyz=contours_xyz_wrt_wholebrain_volResol, 
                               interpolation_direction='z',
                              return_origin_instead_of_bbox=True)

t = time.time()
mesh = volume_to_polydata(vol, origin=origin_wrt_wholebrain_volResol, 
                              num_simplify_iter=3, smooth=True)

sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

# fp = DataManager.get_mesh_filepath(stack_m=stack, vol_type_m='annotation', structure=name_s)
# create_parent_dir_if_not_exists(fp)
# save_mesh_stl(mesh, fp)
# upload_to_s3(fp)

In [ ]:
stack_spec 

In [97]:
DataManager.get_original_volume_filepath_v2(stack_spec=dict(name=stack, resolution='10.0um', 
                                                            vol_type='annotation', 
                                                            prep_id=None,
                                                            detector_id=None,
                                                           structure=name_s), 
                                            structure=name_s)

TypeError: exceptions must be old-style classes or derived from BaseException, not NoneType

In [83]:
wrt_metadata = {'wholebrain': {'origin_wrt_wholebrain_um': 0, 
                'plane': 'sagittal', 
                'zdim_um': None}}

In [85]:
def convert_resolution(p, in_resolution, out_resolution, 
                       stack=None, image_resolution=None, 
                       section_list=None,
                      volume_resolution_um=None):

    if in_resolution == 'image':
        p_um = p * convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
    elif in_resolution == 'image_image_section':
        uv_um = p[..., :2] * convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
        d_um = np.array([SECTION_THICKNESS * sec for sec in p[..., 2]])
        p_um = np.column_stack([uv_um, d_um])
    elif in_resolution == 'image_image_index':
        uv_um = p[..., :2] * convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
        i_um = np.array([SECTION_THICKNESS * section_list[int(idx)] for idx in p[..., 2]])
        p_um = np.column_stack([uv_um, i_um])
    elif in_resolution == 'volume':
        p_um = p * volume_resolution_um
    elif in_resolution == 'raw':
        p_um = p * planar_resolution[stack]
    elif in_resolution == 'down32':
        p_um = p * (planar_resolution[stack] * 32.)
    elif in_resolution == 'um':
        p_um = p
    else:
        raise

    if out_resolution == 'image':
        p_outResol = p_um / convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
    # elif out_resolution == 'image_image_section':
    #     uv_outResol = p_um[..., :2] / convert_resolution_string_to_voxel_size(stack=self.gui.stack, resolution=self.data_feeder.resolution)
    #     sec_outResol = np.array([1 + int(np.floor(d_um / SECTION_THICKNESS)) for d_um in p_um[..., 2]])
    #     p_outResol = np.column_stack([uv_outResol, sec_outResol])
    elif out_resolution == 'image_image_index':
        uv_outResol = p_um[..., :2] / convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
        if hasattr(self.data_feeder, 'sections'):
            i_outResol = []
            for d_um in p_um[..., 2]:
                sec = 1 + int(np.floor(d_um / SECTION_THICKNESS))
                if sec in section_list:
                    index = section_list.index(sec)
                else:
                    index = np.nan
                i_outResol.append(index)
            i_outResol = np.array(i_outResol)
            # i_outResol = np.array([self.data_feeder.sections.index(1 + int(np.floor(d_um / SECTION_THICKNESS))) for d_um in p_um[..., 2]])
        else:
            i_outResol = p_um[..., 2] / convert_resolution_string_to_um(stack=stack, resolution=image_resolution)
        p_outResol = np.column_stack([uv_outResol, i_outResol])
    elif out_resolution == 'volume':
        p_outResol = p_um / volume_resolution_um
    elif out_resolution == 'raw':
        p_outResol = p_um / planar_resolution[stack]
    elif out_resolution == 'down32':
        p_outResol = p_um / (planar_resolution[stack] * 32.)
    elif out_resolution == 'um':
        p_outResol = p_um
    else:
        raise

    return p_outResol

In [86]:
def convert_from_wholebrain_um(p_wrt_wholebrain_um, wrt, resolution,
                               image_resolution=None, stack=None, volume_resolution_um=None,
    structure_origin=None, structure_wrt=None, structure_resolution=None, structure_zdim=None):

    p_wrt_wholebrain_um = np.array(p_wrt_wholebrain_um)
    assert np.atleast_2d(p_wrt_wholebrain_um).shape[1] == 3

    if wrt == 'wholebrain':
        p_wrt_outdomain_um = p_wrt_wholebrain_um
    elif 'sagittal' in wrt or 'coronal' in wrt or 'horizontal' in wrt:
        box, plane = wrt.split('_')
        p_wrt_boxSagittal_origin_um = p_wrt_wholebrain_um - wrt_metadata[wrt]['origin_wrt_wholebrain_um']
        assert plane == 'sagittal', plane # otherwise, need to provide zdim to convert_frame.
        p_wrt_outdomain_um = convert_frame(p_wrt_boxSagittal_origin_um, in_frame='sagittal', 
                                           out_frame=plane, 
                                           zdim=wrt_metadata[wrt]['zdim_um'])
    else:
        print wrt
        raise

    p_wrt_outdomain_outResol = convert_resolution(p_wrt_outdomain_um, in_resolution='um', out_resolution=resolution,
                                                 image_resolution=image_resolution, stack=stack, volume_resolution_um=volume_resolution_um)
    return np.squeeze(p_wrt_outdomain_outResol)

In [87]:
def convert_to_wholebrain_um(p, wrt, resolution,
                             image_resolution=None, stack=None, volume_resolution_um=None,
    structure_origin=None, structure_wrt=None, structure_resolution=None, structure_zdim=None):

    p = np.array(p)
    assert np.atleast_2d(p).shape[1] == 3

    p_um = convert_resolution(p, in_resolution=resolution, out_resolution='um',
                             image_resolution=image_resolution, stack=stack, volume_resolution_um=volume_resolution_um)

    if wrt == 'wholebrain':
        p_wrt_wholebrain_um = p_um
    elif 'sagittal' in wrt or 'coronal' in wrt or 'horizontal' in wrt:
        box, plane = wrt.split('_')
        assert plane == 'sagittal', plane # otherwise, need to provide zdim to convert_frame.
        p_wrt_boxSagittal_um = convert_frame(p_um, in_frame=plane, out_frame='sagittal', zdim=wrt_metadata[wrt]['zdim_um'])
        box_origin_wrt_wholebrain_um = wrt_metadata[wrt]['origin_wrt_wholebrain_um']
        p_wrt_wholebrain_um = p_wrt_boxSagittal_um + box_origin_wrt_wholebrain_um
    else:
        print wrt
        raise

    return np.squeeze(p_wrt_wholebrain_um)

In [88]:
def convert_frame_and_resolution(p, in_wrt, in_resolution, out_wrt, out_resolution,
                                 image_resolution=None, stack=None, volume_resolution_um
    structure_origin=None, structure_wrt=None, structure_resolution=None, structure_zdim=None):
        """
        `wrt` can be any of:
        - wholebrain
        - sagittal: frame of lo-res sagittal scene = sagittal frame of the intensity volume, with origin at the most left/rostral/dorsal position.
        - coronal: frame of lo-res coronal scene = coronal frame of the intensity volume, with origin at the most left/rostral/dorsal position.
        - horizontal: frame of lo-res horizontal scene = horizontal frame of the intensity volume, with origin at the most left/rostral/dorsal position.

        `resolution` can be any of:
        - raw
        - down32
        - vol
        - image: gscene resolution, determined by data_feeder.resolution
        - image_image_index: (u in image resolution, v in image resolution, i in terms of data_feeder index)
        """

        p_wrt_wholebrain_um = convert_to_wholebrain_um(p, wrt=in_wrt, resolution=in_resolution,
                                                       image_resolution=image_resolution, stack=stack, volume_resolution_um=volume_resolution_um,
        structure_origin=structure_origin, structure_wrt=structure_wrt, structure_resolution=structure_resolution, structure_zdim=structure_zdim)

        p_wrt_outdomain_outResol = convert_from_wholebrain_um(p_wrt_wholebrain_um=p_wrt_wholebrain_um, wrt=out_wrt, resolution=out_resolution,
                                                              image_resolution=image_resolution, stack=stack, volume_resolution_um=volume_resolution_um,
        structure_origin=structure_origin, structure_wrt=structure_wrt, structure_resolution=structure_resolution, structure_zdim=structure_zdim)
        # print 'p', p
        # print "p_wrt_wholebrain_um", p_wrt_wholebrain_um
        # print 'p_wrt_outdomain_outResol', p_wrt_outdomain_outResol
        return p_wrt_outdomain_outResol

# Convert integer-valued annotation volume to structure meshes

In [6]:
volume = DataManager.load_annotation_volume(stack=stack, downscale=32)

In [20]:
meshes = {}

for name_s, label in name_to_label.iteritems():
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume == label) 
    print name_s, xmin, xmax, ymin, ymax, zmin, zmax
    
    vol = (volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == label).astype(np.float)

    t = time.time()
    mesh = volume_to_polydata(vol, origin=(ann_xmin+xmin, ann_ymin+ymin, ann_zmin+zmin), 
                                  num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5
    
    meshes[name_s] = mesh
    
    fp = DataManager.get_mesh_filepath(stack_m=stack, vol_type_m='annotation', structure=name_s)
    create_parent_dir_if_not_exists(fp)
    save_mesh_stl(mesh, fp)
    upload_to_s3(fp)

LRt_R 306 402 238 292 236 287


marching cube: 0.05 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 13651: 0.15 seconds
simplify 1 @ 2724: 0.11 seconds
simplify 2 @ 546: 0.02 seconds
time: 0.35


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_LRt_R.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_LRt_R.stl"
Sp5C_L 351 515 175 298 58 137


marching cube: 0.22 seconds
mesh_to_polydata: 0.05 seconds
simplify 0 @ 38592: 0.64 seconds
simplify 1 @ 7720: 0.66 seconds
simplify 2 @ 1545: 0.14 seconds
time: 1.73


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_Sp5C_L.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_Sp5C_L.stl"
Pn_L 82 155 267 320 98 168


...

marching cube: 0.68 seconds
mesh_to_polydata: 0.15 seconds
simplify 0 @ 125499: 1.13 seconds
simplify 1 @ 25101: 1.41 seconds
simplify 2 @ 5021: 0.29 seconds
time: 3.68


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_SC.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_SC.stl"
VCP_L 231 282 157 244 15 46
aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_VCP_L.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_VCP_L.stl"


marching cube: 0.02 seconds
mesh_to_polydata: 0.01 seconds
simplify 0 @ 6754: 0.08 seconds
simplify 1 @ 1352: 0.05 seconds
simplify 2 @ 272: 0.01 seconds
time: 0.17


Amb_L 293 327 243 255 99 106
aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_Amb_L.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_down32_annotationVolume/MD594_down32_annotationVolume_Amb_L.stl"


marching cube: 0.00 seconds
mesh_to_polydata: 0.00 seconds
simplify 0 @ 613: 0.01 seconds
simplify 1 @ 124: 0.00 seconds
time: 0.02


# Load 

In [10]:
meshes_rel2uncropped = DataManager.load_meshes(stack_m='MD589', vol_type_m='annotation', sided=True)

In [18]:
mesh_actors_rel2uncropped = [actor_mesh(mesh_rel2uncropped, np.array(name_unsided_to_color[convert_to_original_name(name_s)])/255., 
                          wireframe=False, opacity=1.) 
               for name_s, mesh_rel2uncropped in meshes_rel2uncropped.iteritems()]

In [21]:
shell_polydata_rel2MD589uncropped = DataManager.load_shell_mesh(stack='MD589', downscale=32)
shell_actor_rel2MD589uncropped = actor_mesh(shell_polydata_rel2MD589uncropped, (1,1,1), opacity=.1, wireframe=False)

In [22]:
launch_vtk(mesh_actors_rel2uncropped + [shell_actor_rel2MD589uncropped], init_angle='45')

In [ ]:
fn = os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack})
outerContour_polydata = load_mesh_stl(fn, return_polydata_only=True)

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.1, wireframe=False)

In [ ]:
launch_vtk(mesh_actors + [outerContour_actor], init_angle='30', background_color=(0,0,0))

In [ ]:
# Export to rgb PLY file (STL file can not have color info)
# https://github.com/Kitware/VTK/blob/master/IO/PLY/Testing/Python/TestPLYReadWrite.py#L31
# http://www.vtk.org/doc/nightly/html/classvtkPLYWriter.html#aa7f0bdbb2decdc7a7360a890a6c10e8b

for name_s, polydata in polydata_list.iteritems():
    fn = os.path.join(mesh_rootdir, stack, "%(stack)s_down32_%(name)s_smoothed.ply" % {'stack': stack, 'name':name_s})
    save_mesh(polydata, fn, color=(colors[convert_name_to_unsided(name_s)]*255).astype(np.int))